<a href="https://colab.research.google.com/github/silviolima07/PUC_residencia_tec_ml/blob/main/Aula20_Assincrona_AtividadePratica03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Grupo:
- Silvio Cesar de Lima - RA 23029232
- Vitor Pereira Andrade - RA 23029249
- Marcos Paulo Cassiano Pena - RA 23029256

# Aula 20 - Assícrona - Atividade Prática 03

##  Implementação de modelos e verificação de eficiência da estratégia adotada em problemas de classificação considerando diferentes métodos de engenharia de features

Esta atividade corresponde à nossa terceira atividade **valorizada** do curso de *Machine Learning*.

### Instruções:

1 - Reunam-se em grupos de **no máximo 3 pessoas**;

2 - A entrega deve ser realizada via CANVAS através de um artigo no **formato PDF gerado a partir do *notebook***. Para tal fim, os grupos poderão utilizar o *Jupyter Notebook* ou o *Google Drive* para implementação do que foi solicitado.

3 - O período de disponibilidade para recebimento da atividade será de **25/09/23 às 17h, até 02/10/23, às 17h**.

4 - No cabeçalho do documento devem constar o **nome completo e o RA de TODOS os membros do grupo**.

#### Práticas a serem desenvolvidas:

1 - Substituição de valores inválidos;

2 - Verificação de eficiência considerando técnicas de normalização;

3 - Seleção de *Features*;

4 - Extração de *Features*;

5 - Validação Cruzada considerando o método *k-fold*.

### Parâmetros fixos a serem utilizados:

1 - Classificador ***Random Forest*** com `random_state = 0`;

2 - Função `train_test_split`:

     - Proporção de divisão de dados de treinamento e teste: 70% e 30%, respectivamente;
     - `random_state = 42`.
     


**Questão 1)** Considere o banco de dados `water_potability.csv`.

**a)** Verifique a existência de valores inválidos nos dados de entrada do *dataset* e, caso existam, observe os seguintes pontos:

    - Se a porcentagem de dados inválidos for maior do que 20%, exclua esta(s) coluna(s);
    - Se a porcentagem de dados inválidos for inferior a 20%, substitua os valores inválidos pelo valor imediatamente anterior, o método `bfill`.
    
**b)** Normalize os dados de entrada através do método `MinMaxScaler`.

**c)** Realize a tarefa de classificação utilizando ***Random Forest***, considerando 10 árvores na floresta.

**d)** Verifique a eficiência do método empregado utilizando a função `classification_report`.

**Questão 2)** Considerando os dados normalizados, e o modelo do classificador criado no exercício anterior, utilize o método de **seleção de features** **RFE** (*Recursive Feature Elimination*) para selecionar **3** *features* que farão parte da tarefa de classificação. Identifique quais colunas foram selecionadas. Analise os resultados por meio da função `classification_report`.

**Questão 3)** Considerando os dados normalizados e o modelo do classificador criado na **Questão 1)**, utilize a **Análise de Componentes Principais** para extrair as *features* que farão parte da tarefa de classificação. Considere utilizar **3** componentes principais e analise os resultados por meio da função `classification_report`.

**Questão 4)** Considerando as componentes principais extraídas na **Questão 3)**, utilize o método de **Validação Cruzada** por ***k-fold*** para verificar o número de àrvores adequado para a floresta na solução do problema. Considere variar o número de árvores entre 10 e 100, com passo de 10. Considere também `k = 10`, no método *k-fold*. Plote o resultado da acurácia do modelo de acordo com o número de árvores na floresta e realize a classificação considerando o melhor resultado alcançado na etapa de validação.

**Questão 5)** Explique os resultados obtidos nos exercícios anteriores.